In [ ]:
import numpy as np
import igraph as ig
import itertools as it
from scipy.special import logsumexp

# Gibbs sampling inference algorithm

In [ ]:
def gibbs_fast(networks,K,num_samples = 100,skip = 10,n_restarts = 1,burn_in=1,\
                  alpha_guess = None,beta_guess = None,\
                  pi_guess = None,rho_guess = None,
                 labels_guess = None,beta_params = [1.,1.,1.,1.,1.,1.],\
                 fix_labels = False,fix_theta = False,\
                  invert = False,fix_rho=False,fix_modes=False):
    
    """
    inputs:
        networks: list of igraph graph objects representing the population to be clustered
        num_samples: number of Gibbs samples to perform
        skip: interval at which to take samples as algorithm runs
        n_restarts: number of random restarts
        burn_in: thermalization period for Gibbs sampler
        alpha_guess: guess for alphas (length K array); used to initialize sampling
        beta_guess: guess for betas (length K array); used to initialize sampling
        pi_guess: guess for pis (length K array); used to initialize sampling
        rho_guess: guess for rho (scalar); used to initialize sampling
        labels_guess: guess for cluster assignments (length n array); used to initialize sampling
        beta_params: beta prior parameters; [Hu11,Hu01,Hu10,Hu00,astar,bstar] variables from paper
        fix_labels: whether or not to fix the cluster assignments as those specified in 'labels_guess'
        fix_theta: whether or not to fix the alpha/beta/pi/rho parameters as those specified in the corresponding '_guess' parameters
        invert: whethor or not to allow alpha,beta parameters to exceed 0.5 (corresponding to completely random edge flips)
        fix_rho: whether or not to fix rho parameter from rho_guess only
        fix_modes: 'False' indicates not fixing modes, and otherwise input needs to be dictinary of adjacency matrix arrays for modes
        
    returns (in order):
        posterior mean mode estimators, given by dict of 2D n x n arrays representing adjacency matrices of modes
        posterior mode cluster assignments, given by length n array of cluster labels for each sample network
        posterior mean theta parameter estimatrs, given by concatenated length 3K vector [alpha_hat,beta_hat,pi_hat]
        posterior mean estimator for mode density, given by scalar
        mean log probability over samples, given by scalar
    """
    
    """
    initialize constants and dictionaries for comparing edge lists quickly
    """
    n = networks[0].vcount()
    ms = [g.ecount() for g in networks]
    N = len(networks)
    nc2 = n*(n-1)/2
    edge_dicts = {}
    for t,g in enumerate(networks):
        edge_dicts[t] = {}
        for e in g.es:
            edge_dicts[t][tuple(sorted(e.tuple))] = 1
    Hu11,Hu01,Hu10,Hu00,astar,bstar = beta_params
    
    """
    map single index to (i,j) value of upper triangle of adjacency matrix
    used for quick edge flip sampling in mode update step
    """
    def ind2ij(ind,n):
        i = n - 2 - np.floor(np.sqrt(-8*ind + 4*n*(n-1)-7)/2.0 - 0.5)
        j = ind + i + 1 - n*(n-1)/2 + (n-i)*((n-i)-1)/2
        return i,j
    
    
    """
    main loop
    """
    As,gs,thetas,rhos,logPs = [],[],[],[],[]
    for restart in range(n_restarts):
        
        """
        randomize each parameter according to its prior if no guesses are given
        """
        if alpha_guess is None: alpha_guess = np.random.beta(Hu11,Hu01,size=K)
        if beta_guess is None: beta_guess = np.random.beta(Hu10,Hu00,size=K)
        if pi_guess is None: pi_guess = np.ones(K)/K
        if rho_guess is None: rho_guess = np.random.beta(astar,bstar)
        if labels_guess is None: labels_guess = np.random.choice(np.arange(K),size=N,p=pi_guess)
        if fix_modes != False:
            A = fix_modes.copy()
            
        alpha = alpha_guess.copy()
        beta = beta_guess.copy()
        pi = pi_guess.copy()
        rho = rho_guess
        g = labels_guess.copy()
        
        """
        initialize dict of adjacency matrix arrays for modes, array for cluster labels, arrays for theta parameters
        the averages of these over all samples in the restart will be obtained as posterior mean estimates of A and theta
        argmaxes over samples will be taken for Z estimator
        """
        alpha_hat,beta_hat,pi_hat,rho_hat = np.zeros(K),np.zeros(K),np.zeros(K),0.
        A_hat,g_hat = dict.fromkeys(np.arange(K)),[]
        for u in range(K):
            A_hat[u] = {}
        logP_hat = 0.
        
        """
        initialize X and N arrays based on initial cluster assignments
        """
        X,Ns = {},np.zeros(K)
        for u in range(K):
            X[u] = {}
            Ns[u] = 0.
            for t in range(N):
                if g[t] == u:
                    for e in edge_dicts[t]:
                        if e in X[u]: X[u][e] += 1.
                        else: X[u][e] = 1.
                    Ns[u] += 1.
        Xinv = {}
        for u in range(K):
            Xinv[u] = {}
            for k, v in X[u].items():
                Xinv[u][v] = Xinv[u].get(v, []) + [k]
        
        
        """
        take Gibbs samples
        """
        num_sampled = 0.
        for samp in range(num_samples*skip + burn_in):
            
            """
            sample A if modes are not fixed
            uses accelerated sampling scheme described in paper 
            """
            if fix_modes == False:

                A = {}
                for u in range(K):
                    A[u] = {}
                    total_nonzero = 0
                    for l in Xinv[u].keys():
                        logdt = np.log(1.-rho) - np.log(rho) + l*np.log(beta[u]) - l*np.log(alpha[u])\
                                + (Ns[u]-l)*np.log(1-beta[u]) - (Ns[u]-l)*np.log(1-alpha[u])
                        Q = 1./(1. + np.exp(logdt))
                        total_in_class = len(Xinv[u][l])
                        total_nonzero += total_in_class
                        num2pick = np.random.binomial(total_in_class,p=Q)
                        for _ in range(num2pick):
                            e = Xinv[u][l].pop()
                            A[u][e] = 1
                    logdt = np.log(1.-rho) - np.log(rho) \
                                + Ns[u]*np.log(1-beta[u]) - Ns[u]*np.log(1-alpha[u])
                    Q = 1./(1. + np.exp(logdt))
                    total_zero = nc2 - total_nonzero
                    num2pick = np.random.binomial(total_zero,p=Q)
                    num_picked = 0
                    num_fails = 0
                    while (num_picked < num2pick) and (num_fails < 100):
                        ind = np.random.randint(int(nc2))
                        i,j = ind2ij(ind,n)
                        i,j = int(i),int(j)
                        if not((i,j) in X[u]) and (i != j): 
                            A[u][(i,j)] = 1
                            num_picked += 1
                            num_fails = 0
                        else:
                            num_fails += 1
            
            """
            update Y array based on results from A sampling
            """
            Y = {}
            for u in range(K):
                Y[u] = {}
                for t in range(N):
                    Y[u][t] = {'11':0,'10':0,'01':0,'00':0}
                    for e in A[u].keys():
                        if e in edge_dicts[t]: 
                            Y[u][t]['11'] += 1.
                        else:
                            Y[u][t]['01'] += 1.
                    Y[u][t]['10'] = ms[t] - Y[u][t]['11']
                    Y[u][t]['00'] = nc2 - Y[u][t]['11'] - Y[u][t]['01'] - Y[u][t]['10']       
            
            """
            update W array and Mstar based on Y array
            """
            W,Mstar = {},0.
            for u in range(K):
                W[u] = {'11':0,'10':0,'01':0,'00':0}
            for t in range(N):
                u = g[t]
                for s in ['11','01','10','00']:
                    W[u][s] += Y[u][t][s]
            for u in range(K):
                Mstar += (W[u]['11']+W[u]['01'])/(Ns[u]+1e-100)
            
            """
            sample theta parameters if not fixed
            """
            if fix_theta == False:
                
                for u in range(K):
                    alpha[u] = np.random.beta(W[u]['11']+Hu11,W[u]['01']+Hu01)
                    beta[u] = np.random.beta(W[u]['10']+Hu10,W[u]['00']+Hu00)
                if invert == True:
                    for u in range(K):
                        alpha[u] = max(alpha[u],1.-alpha[u])
                        beta[u] = min(beta[u],1.-beta[u])
                    
                pi = np.random.dirichlet(1.+Ns)
                if fix_rho == True:
                    rho = rho_guess
                else:
                    rho = np.random.beta(astar+Mstar,bstar+K*nc2-Mstar)

            """
            sample cluster labels if not fixed
            """
            if fix_labels == False:

                R = {}
                for t in range(N):
                    R[t] = {}
                    logRsum = []
                    for u in range(K):
                        logR = np.log(pi[u]) + Y[u][t]['11']*np.log(alpha[u])\
                                + Y[u][t]['01']*np.log(1-alpha[u])\
                                + Y[u][t]['10']*np.log(beta[u])\
                                + Y[u][t]['00']*np.log(1-beta[u])
                        R[t][u] = logR
                        logRsum.append(logR)
                    logRsum = logsumexp(logRsum)
                    for u in range(K):
                        R[t][u] = R[t][u] - logRsum
                    g[t] = np.random.choice(np.arange(K),p=[np.exp(R[t][u]) for u in range(K)])

                    
            """
            update X and N data structures after cluster assignment sampling
            """
            X,Ns = {},np.zeros(K)
            for u in range(K):
                X[u] = {}
                Ns[u] = 0.
                for t in range(N):
                    if g[t] == u:
                        for e in edge_dicts[t]:
                            if e in X[u]: X[u][e] += 1.
                            else: X[u][e] = 1.
                        Ns[u] += 1.
            Xinv = {}
            for u in range(K):
                Xinv[u] = {}
                for k, v in X[u].items():
                    Xinv[u][v] = Xinv[u].get(v, []) + [k]
            
            """
            take Gibbs sample and update posterior estimators
            """
            if (samp % skip == 0) and (samp > burn_in):
                
                for u in range(K):
                    for e in A[u].keys():
                        if e in A_hat[u]: A_hat[u][e] += 1.
                        else: A_hat[u][e] = 1.
                g_hat.append(g)
                alpha_hat += alpha
                beta_hat += beta
                pi_hat += pi
                rho_hat += rho
                delta_logP_hat = Mstar*np.log(rho) + (K*nc2-Mstar)*np.log(1.-rho)
                for u in range(K):
                    delta_logP_hat += Ns[u]*np.log(pi[u]) + W[u]['11']*np.log(alpha[u])\
                                        + W[u]['01']*np.log(1.-alpha[u]) \
                                        + W[u]['10']*np.log(beta[u])\
                                        + W[u]['00']*np.log(1.-beta[u]) 
                logP_hat += delta_logP_hat
                num_sampled += 1.
                
        """
        compute final posterior estimators and finish restart
        """
        for u in range(K):
            Amat = np.zeros((n,n))
            for e in A_hat[u].keys():
                A_hat[u][e] = A_hat[u][e]/num_sampled
                Amat[e[0],e[1]] = A_hat[u][e]
                Amat[e[1],e[0]] = A_hat[u][e]
            A_hat[u] = Amat
        g_hat = np.array(g_hat)
        g_hat = [np.bincount(g_hat[:,i]).argmax() for i in range(N)]
        theta_hat = np.concatenate([alpha_hat/num_sampled,beta_hat/num_sampled,\
                                    pi_hat/num_sampled])
        rho_hat /= num_sampled
        logP_hat /= num_sampled
        
        As.append(A_hat)
        gs.append(g_hat)
        thetas.append(theta_hat)
        rhos.append(rho_hat)
        logPs.append(logP_hat)
    
    """
    pick restart with highest average log probability and return posterior estimators for that restart
    """
    best_restart = np.argmax(logPs)
    
    return As[best_restart],gs[best_restart],thetas[best_restart],rhos[best_restart],logPs[best_restart]

# generate sample networks from model

In [ ]:
def generate_synthetic(num,modes,alphas,betas,pis):
    
    """
    generate networks from model
    
    inputs:
        num: number of sample networks to be drawn
        modes: igraph graph objects representing mode networks
        alphas: true positive rates for each cluster
        betas: false positive rates for each cluster
        pis: cluster assignment probabilities for each cluster
        
    returns (in order):
        list of igraph graph objects representing sampled networks
        cluster assignments of each sampled network (which mode each was drawn from)
    """
    
    
    mode_As = [np.array(g.get_adjacency().data) for g in modes]
    K = len(modes)
    n = modes[0].vcount()
    gs,cluster_labels = [],[]
    for t in range(num):
        A = np.zeros((n,n))
        u = np.random.choice(np.arange(K),p=pis)
        Aref,alpha,beta = mode_As[u],alphas[u],betas[u]
        cluster_labels.append(u)
        for pair in list(it.combinations(np.arange(n),2)):
            i,j = pair
            r = np.random.rand()
            A[i,j] = (Aref[i,j] == 1)*(r < alpha) + (Aref[i,j] == 0)*(r < beta)
            A[j,i] = A[i,j]
        g = ig.Graph().Adjacency(A.tolist()).as_undirected()
        gs.append(g)
        
    return gs,cluster_labels  